In [1]:
# 데이터가 부족한 경우 사용
# NaN값을 제거하면 데이터가 5302개이므로 클롤링을 통해 NaN값을 채울려고 한다.
import pandas as pd
import numpy as np

- 데이터 분석(감독)

In [10]:
# title budget genres country main_charactor director vote_average vote_count
df = pd.read_csv('data/movie_data_original.csv',
                 names="title postpath openDate budget genres country main_charactor director time overview vote_average vote_count".split(),
                 skiprows=1)

In [11]:
df.head(1)

,title,postpath,openDate,budget,genres,country,main_charactor,director,time,overview,vote_average,vote_count
0,{616037: '토르: 러브 앤 썬더'},https://image.tmdb.org/t/p/original/bZLrpWM065...,2022-07-06,250000000,"[{'id': 28, 'name': '액션'}, {'id': 12, 'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},119,"이너피스를 위해 자아 찾기 여정을 떠난 천둥의 신 토르. 그러나, 우주의 모든 신들...",6.778,1802


In [35]:
# 예산이 표기된 api가 없다...일단 제거
NoBudget = df[df['budget'] == 0]
df = df.drop(NoBudget.index)
len(df)

5142

In [12]:
# 감독이 없는 데이터 114개
NoDirectors = df[df['director'] == '-']
len(NoDirectors)

119

In [14]:
NoDirectors.head(1)

,title,postpath,openDate,budget,genres,country,main_charactor,director,time,overview,vote_average,vote_count
169,{975117: 'Padre no hay mas que Uno 3'},https://image.tmdb.org/t/p/original/k0NLfgdPck...,NaN,0,[],[],{1114487: 'Hero Fiennes Tiffin'},-,0,NaN,6.8,3


In [15]:
import ast

In [16]:
# 감독이 없는 리스트 중 타이틀을 string -> dict로 그중 title name만 가져옴
# Ndirector 에 리스트로 저장
Ndirector = []
for NoDirector in df.loc[NoDirectors.index].title:
    for s in ast.literal_eval(NoDirector).values():
        Ndirector.append(s)

In [17]:
Ndirector

['Padre no hay mas que Uno 3',
 "Chip and Potato: Chip's Holiday",
 'The Monster at the End of This Story',
 'Dragon Ball Z: The Real 4-D at 超天下一武道会',
 '수퍼 소닉 3',
 '착한 엄마친구 2',
 'ドラゴンボール Ｚ あつまれ！ 悟空ワールド',
 'Depredador',
 'Naruto to Boruto: The Live 2019',
 '룸싸롱 : 맛있는 서비스2',
 'Ninja Escapades: Creation of Akatsuki, The Two Uchiha, The Far Reaches of Hope',
 'Misterio en Marbella',
 'Your Name',
 '粉色樱与大眠王',
 '어벤져스: 시크릿 워즈',
 'La Vida en el Silencio',
 '미용실 : 특별한 서비스 4',
 "This Isn't Sex Tape XXX: It's A Spoof!",
 'Avatar：Production Materials',
 'Boruto: Jump Festa Special',
 '50 Greatest Harry Potter Moments',
 'Moana Reo Māori',
 'Los 7 pecados capitales',
 '뛰뛰빵빵! 코리의 모험 - 달려라! 크리시',
 "James Corden's Top Gun Training with Tom Cruise",
 'Avatar: Scene Deconstruction',
 'The Boys - Behind the Scenes: Insider Look',
 '착한 형수2',
 '베놈 3',
 '슈퍼 몬스터의 신비한 옛날이야기',
 '원피스 - 대양에 펼쳐라! 크나큰 아빠의 꿈!',
 'Predators: Moments of Extraction',
 'Naruto Shippuden: OVA Hashirama Senju vs Madara Uchiha',
 'Constan

- 클롤링(감독 데이터)

In [18]:
# 네이버 api에서 제목 검색 => 
import requests
import json

In [21]:
with open('APIKey/Naver.json') as file:
    api = json.load(file)
client_id = api['id']
client_secret = api['secret']

In [33]:
search = "Chip and Potato: Chip's Holiday"
url = "https://openapi.naver.com/v1/search/movie.json"
headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret
           }
params = {
    'query' : '[극장판] 레이디버그: 미라클스톤의 비밀'
}
response = requests.get(url, headers=headers, params=params)
response

<Response [200]>

In [34]:
result = json.loads(response.text)
result

{'lastBuildDate': 'Wed, 17 Aug 2022 11:06:51 +0900',
 'total': 1,
 'start': 1,
 'display': 1,
 'items': [{'title': '<b>극장판</b> <b>레이디</b><b>버그</b>: <b>미라클</b><b>스톤</b>의 <b>비밀</b>',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=171727',
   'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1717/171727_P14_110648.jpg',
   'subtitle': 'Miraculous seasons 2',
   'pubDate': '2017',
   'director': '토마스 아스트뤽|',
   'actor': '여민정|남도형|홍시호|',
   'userRating': '9.39'}]}

In [ ]:
with open('APIKey/Imdb.json') as file:
    key = json.loads(file)['key']
url = f"https://imdb-api.com/ko/API/Title/{key}/tt1375666"
